# Evaluation Pipeline Testing Notebook
## Reasoning Distillation Project

This notebook tests:
1. Individual metrics (Accuracy, ROUGE, BERTScore, Faithfulness)
2. Full evaluation pipeline
3. Error analysis and confusion matrix
4. Per-label evaluation
5. Model comparison
6. Batch evaluation
7. Results visualization

In [ ]:
# Setup
import sys
from pathlib import Path
import warnings
import time
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import pandas as pd

from src.data.data_loader import TeacherDataLoader
from src.data.preprocessor import ReasoningPreprocessor, PreprocessConfig
from src.data.dataset import ESNLIDataset, create_dataloaders

from src.models.student import StudentModel, StudentConfig

from src.evaluation.metrics import (
    LabelAccuracyMetric,
    ROUGEMetric,
    BERTScoreMetric,
    ExplanationFaithfulnessMetric,
    StudentTeacherAgreementMetric,
    compute_all_metrics,
    MetricsConfig,
    format_metrics
)

## 1. Test Individual Metrics

In [ ]:
# Create sample predictions and references
print("=" * 70)
print("TESTING INDIVIDUAL METRICS")
print("=" * 70)

sample_predictions = [
    "entailment explanation: The person is definitely on a horse.",
    "neutral explanation: We cannot determine if they are training.",
    "contradiction explanation: The person cannot be both inside and outside.",
    "entailment explanation: This clearly follows from the premise.",
    "neutral explanation: It's unclear what the intention is."
]

sample_references = [
    "entailment explanation: The premise states a person is on a horse.",
    "neutral explanation: Training is not mentioned in the premise.",
    "contradiction explanation: These are mutually exclusive locations.",
    "entailment explanation: The hypothesis is supported by the premise.",
    "contradiction explanation: The actions contradict each other."  # Mismatch for testing
]

print(f"\nCreated {len(sample_predictions)} test samples")

### 1.1 Test Label Accuracy

In [ ]:
# Test label accuracy metric
print("\n" + "=" * 70)
print("LABEL ACCURACY METRIC")
print("=" * 70)

label_metric = LabelAccuracyMetric()

# Test label extraction
print("\nLabel extraction test:")
for pred in sample_predictions[:3]:
    label = label_metric.extract_label(pred)
    print(f"  '{pred[:40]}...' → {label}")

# Compute accuracy
accuracy_results = label_metric.compute(sample_predictions, sample_references)

print("\nAccuracy results:")
print(format_metrics(accuracy_results))

### 1.2 Test ROUGE Scores

In [ ]:
# Test ROUGE metric
print("\n" + "=" * 70)
print("ROUGE METRIC")
print("=" * 70)

rouge_metric = ROUGEMetric()

if rouge_metric.available:
    # Test explanation extraction
    print("\nExplanation extraction test:")
    for pred in sample_predictions[:2]:
        explanation = rouge_metric.extract_explanation(pred)
        print(f"  Input: {pred}")
        print(f"  Extracted: {explanation}\n")
    
    # Compute ROUGE
    rouge_results = rouge_metric.compute(sample_predictions, sample_references)
    
    print("ROUGE results:")
    print(format_metrics(rouge_results))
else:
    print("\n⚠️  ROUGE not available. Install with: pip install rouge-score")

### 1.3 Test BERTScore

In [ ]:
# Test BERTScore metric
print("\n" + "=" * 70)
print("BERTSCORE METRIC")
print("=" * 70)

bertscore_metric = BERTScoreMetric()

if bertscore_metric.available:
    print("\nComputing BERTScore (this may take a moment)...")
    bertscore_results = bertscore_metric.compute(
        sample_predictions, 
        sample_references,
        batch_size=8
    )
    
    print("\nBERTScore results:")
    print(format_metrics(bertscore_results))
else:
    print("\n⚠️  BERTScore not available. Install with: pip install bert-score")

### 1.4 Test Explanation Faithfulness

In [ ]:
# Test faithfulness metric
print("\n" + "=" * 70)
print("EXPLANATION FAITHFULNESS METRIC")
print("=" * 70)

faithfulness_metric = ExplanationFaithfulnessMetric()

# Test individual faithfulness checks
print("\nFaithfulness checks:")
for pred in sample_predictions:
    label, explanation = faithfulness_metric.extract_label_and_explanation(pred)
    if label is not None:
        is_faithful = faithfulness_metric.check_faithfulness(label, explanation)
        status = "✓" if is_faithful else "✗"
        print(f"  {status} {label}: {explanation[:50]}...")
    else:
        print(f"  ✗ No label found: {explanation[:50]}...")

# Compute overall faithfulness
faithfulness_results = faithfulness_metric.compute(sample_predictions)

print("\nFaithfulness results:")
print(format_metrics(faithfulness_results))

### 1.5 Test All Metrics Combined

In [ ]:
# Compute all metrics at once
print("\n" + "=" * 70)
print("ALL METRICS COMBINED")
print("=" * 70)

metrics_config = MetricsConfig(
    compute_rouge=True,
    compute_bertscore=True,
    compute_faithfulness=True
)

all_metrics = compute_all_metrics(
    sample_predictions,
    sample_references,
    metrics_config
)

print("\nAll metrics:")
print(format_metrics(all_metrics))

## 2. Prepare Test Data and Model

In [ ]:
# Load test data
print("=" * 70)
print("LOADING TEST DATA")
print("=" * 70)

loader = TeacherDataLoader()
esnli_data = loader.load_esnli()

# Use validation set for testing
test_subset = esnli_data['validation'].select(range(50))  # 50 samples for quick test

print(f"\n✓ Test samples: {len(test_subset)}")

In [ ]:
# Create dataset and dataloader
preprocess_config = PreprocessConfig(
    model_name="google/flan-t5-small",
    max_source_length=128,
    max_target_length=64
)

preprocessor = ReasoningPreprocessor(preprocess_config)
test_dataset = ESNLIDataset(test_subset, preprocessor, use_cache=True)

test_loader = create_dataloaders(
    test_dataset,
    batch_size=8,
    num_workers=0,
    pad_token_id=preprocessor.tokenizer.pad_token_id,
    shuffle_train=False
)

print(f"✓ Test batches: {len(test_loader)}")

In [ ]:
# Load model (untrained for testing)
print("\n" + "=" * 70)
print("LOADING MODEL")
print("=" * 70)

student_config = StudentConfig(
    model_name="google/flan-t5-small",
    max_source_length=128,
    max_target_length=64,
    device=device,
    num_beams=4
)

student = StudentModel(student_config)

print(f"\n✓ Model: {student.count_parameters():,} parameters")

## 3. Test Full Evaluation Pipeline

In [ ]:
# Create evaluator
print("=" * 70)
print("INITIALIZING EVALUATOR")
print("=" * 70)

eval_config = EvaluationConfig(
    metrics_config=MetricsConfig(
        compute_rouge=True,
        compute_bertscore=False,  # Skip BERTScore for speed
        compute_faithfulness=True
    ),
    num_beams=4,
    max_length=64,
    save_predictions=True,
    save_detailed_results=True,
    analyze_errors=True,
    num_error_examples=5,
    output_dir="../experiments/evaluation_test"
)

evaluator = Evaluator(student, eval_config)

print("\n✓ Evaluator initialized!")

In [ ]:
# Run evaluation
print("\n" + "=" * 70)
print("RUNNING FULL EVALUATION")
print("=" * 70)

results = evaluator.evaluate(test_loader, split_name="test")

print("\n✓ Evaluation complete!")

In [ ]:
# Display results
print("\n" + "=" * 70)
print("EVALUATION RESULTS")
print("=" * 70)

metrics = results['metrics']
print("\nMetrics:")
print(format_metrics(metrics))

print(f"\nGenerated {len(results['predictions'])} predictions")
print(f"Evaluation time: {metrics['eval_time_seconds']:.2f}s")
print(f"Throughput: {metrics['samples_per_second']:.2f} samples/sec")

## 4. Analyze Predictions

In [ ]:
# Display sample predictions
print("=" * 70)
print("SAMPLE PREDICTIONS")
print("=" * 70)

predictions = results['predictions']
references = results['references']
inputs = results['inputs']

for i in range(min(3, len(predictions))):
    print(f"\n{'='*70}")
    print(f"SAMPLE {i+1}")
    print(f"{'='*70}")
    print(f"\nInput:\n{inputs[i]}")
    print(f"\nReference:\n{references[i]}")
    print(f"\nPrediction:\n{predictions[i]}")

## 5. Error Analysis

In [ ]:
# Display error analysis
print("=" * 70)
print("ERROR ANALYSIS")
print("=" * 70)

error_analysis = results['error_analysis']

if error_analysis:
    print(f"\nTotal errors: {error_analysis['total_errors']}")
    print(f"Error rate: {error_analysis['error_rate']:.2%}")
    
    # Display confusion matrix
    print("\n" + "=" * 70)
    print("CONFUSION MATRIX")
    print("=" * 70)
    
    confusion = error_analysis['confusion_matrix']
    
    # Create DataFrame for better display
    labels = ['entailment', 'neutral', 'contradiction']
    confusion_df = pd.DataFrame(
        [[confusion[true_label][pred_label] for pred_label in labels]
         for true_label in labels],
        index=labels,
        columns=labels
    )
    
    print("\n", confusion_df)
    
    # Sample errors
    print("\n" + "=" * 70)
    print("SAMPLE ERRORS")
    print("=" * 70)
    
    for i, error in enumerate(error_analysis['sample_errors'][:3], 1):
        print(f"\n--- Error {i} ---")
        print(f"Predicted: {error['predicted_label']}")
        print(f"True: {error['true_label']}")
        print(f"Input: {error['input'][:100]}...")
        print(f"Prediction: {error['prediction'][:100]}...")
else:
    print("\nNo error analysis available")

In [ ]:
# Visualize confusion matrix
if error_analysis:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Create heatmap
    sns.heatmap(
        confusion_df,
        annot=True,
        fmt='d',
        cmap='Blues',
        square=True,
        cbar_kws={'label': 'Count'},
        ax=ax
    )
    
    ax.set_xlabel('Predicted Label', fontsize=12)
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 6. Per-Label Evaluation

In [ ]:
# Evaluate performance by label
print("=" * 70)
print("PER-LABEL EVALUATION")
print("=" * 70)

per_label_metrics = evaluator.evaluate_by_label(test_loader)

print("\n✓ Per-label evaluation complete!")

In [ ]:
# Visualize per-label accuracy
if per_label_metrics:
    labels = list(per_label_metrics.keys())
    accuracies = [per_label_metrics[label].get('label_accuracy', 0) 
                  for label in labels]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = ['#2ecc71', '#3498db', '#e74c3c']
    bars = ax.bar(labels, accuracies, color=colors, alpha=0.7, edgecolor='black')
    
    ax.set_ylabel('Accuracy')
    ax.set_title('Label Accuracy by Category')
    ax.set_ylim(0, 1.0)
    ax.axhline(y=float(np.mean(accuracies)), color='red', linestyle='--', 
               label=f'Mean: {np.mean(accuracies):.3f}')
    
    # Add value labels
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{acc:.3f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    ax.legend()
    plt.tight_layout()
    plt.show()

## 7. Compare Metrics Across Labels

In [ ]:
# Create comparison DataFrame
if per_label_metrics:
    print("=" * 70)
    print("METRICS COMPARISON ACROSS LABELS")
    print("=" * 70)
    
    # Extract common metrics
    metric_names = ['label_accuracy', 'rouge1', 'rouge2', 'rougeL', 'faithfulness']
    
    comparison_data = []
    for label in labels:
        row = {'label': label}
        for metric in metric_names:
            row[metric] = float(per_label_metrics[label].get(metric, 0))
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n", comparison_df.to_string(index=False))
    
    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    metrics_to_plot = ['label_accuracy', 'rouge1', 'rougeL', 'faithfulness']
    
    for idx, metric in enumerate(metrics_to_plot):
        values = comparison_df[metric].values
        axes[idx].bar(labels, values, color=colors, alpha=0.7, edgecolor='black')
        axes[idx].set_title(metric.upper().replace('_', ' '))
        axes[idx].set_ylim(0, 1.0)
        
        # Add values
        for i, v in enumerate(values):
            axes[idx].text(i, v + 0.02, f'{v:.3f}', 
                          ha='center', fontsize=10, fontweight='bold')
    
    plt.suptitle('Metrics Comparison Across Labels', 
                 fontsize=14, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.show()

## 8. Test Quick Evaluate Function

In [ ]:
# Test quick evaluate helper
print("=" * 70)
print("TESTING QUICK EVALUATE")
print("=" * 70)

quick_metrics = quick_evaluate(
    student,
    test_loader,
    output_dir="../experiments/quick_eval_test"
)

print("\nQuick evaluation metrics:")
print(format_metrics(quick_metrics))

## 9. Test Batch Evaluator (Multiple Models)

In [ ]:
# Create batch evaluator
print("=" * 70)
print("TESTING BATCH EVALUATOR")
print("=" * 70)

batch_evaluator = BatchEvaluator(
    output_dir="../experiments/batch_evaluation_test"
)

# Add current model evaluation (flan-t5-small)
print("\nEvaluating Model 1 (FLAN-T5-small)...")
batch_evaluator.add_evaluation(
    "flan-t5-small",
    evaluator,
    test_loader
)

# Create and evaluate flan-t5-base model
print("\nEvaluating Model 2 (FLAN-T5-base)...")

# Initialize base model
student_config_base = StudentConfig(
    model_name="google/flan-t5-base",
    max_source_length=128,
    max_target_length=64,
    device=device,
    num_beams=4
)

student_base = StudentModel(student_config_base)

# Create evaluator for base model
eval_config_base = EvaluationConfig(
    metrics_config=MetricsConfig(
        compute_rouge=True,
        compute_bertscore=False,  # Skip BERTScore for speed
        compute_faithfulness=True
    ),
    num_beams=4,
    max_length=64,
    save_predictions=True,
    save_detailed_results=True,
    analyze_errors=True,
    num_error_examples=5,
    output_dir="../experiments/evaluation_base_test"
)

evaluator_base = Evaluator(student_base, eval_config_base)

# Add base model evaluation
batch_evaluator.add_evaluation(
    "flan-t5-base",
    evaluator_base,
    test_loader
)

# Display batch results
print("\n" + "=" * 70)
print("BATCH EVALUATION RESULTS")
print("=" * 70)

for result in batch_evaluator.results:
    print(f"\nModel: {result['model_name']}")
    print(format_metrics(result['metrics']))


In [ ]:
# Visualize batch results comparison
if len(batch_evaluator.results) > 0:
    print("\n" + "=" * 70)
    print("BATCH RESULTS VISUALIZATION")
    print("=" * 70)
    
    # Extract metrics for comparison
    models = [r['model_name'] for r in batch_evaluator.results]
    accuracy_scores = [r['metrics'].get('label_accuracy', 0) for r in batch_evaluator.results]
    rouge1_scores = [r['metrics'].get('rouge1', 0) for r in batch_evaluator.results]
    faithfulness_scores = [r['metrics'].get('faithfulness', 0) for r in batch_evaluator.results]
    
    # Create comparison visualizations
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Plot 1: Accuracy
    axes[0].bar(models, accuracy_scores, color='#2ecc71', alpha=0.7, edgecolor='black')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title('Label Accuracy Comparison')
    axes[0].set_ylim(0, 1.0)
    for i, v in enumerate(accuracy_scores):
        axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')
    
    # Plot 2: ROUGE1
    axes[1].bar(models, rouge1_scores, color='#3498db', alpha=0.7, edgecolor='black')
    axes[1].set_ylabel('ROUGE1 Score')
    axes[1].set_title('ROUGE1 Comparison')
    axes[1].set_ylim(0, 1.0)
    for i, v in enumerate(rouge1_scores):
        axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')
    
    # Plot 3: Faithfulness
    axes[2].bar(models, faithfulness_scores, color='#e74c3c', alpha=0.7, edgecolor='black')
    axes[2].set_ylabel('Faithfulness Score')
    axes[2].set_title('Explanation Faithfulness Comparison')
    axes[2].set_ylim(0, 1.0)
    for i, v in enumerate(faithfulness_scores):
        axes[2].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')
    
    plt.suptitle('Multi-Model Performance Comparison', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


In [ ]:
# Save batch evaluation report
print("\n" + "=" * 70)
print("SAVING BATCH EVALUATION REPORT")
print("=" * 70)

# Create summary DataFrame
report_data = []
for result in batch_evaluator.results:
    metrics = result['metrics']
    row = {
        'Model': result['model_name'],
        'Accuracy': metrics.get('label_accuracy', 0),
        'ROUGE1': metrics.get('rouge1', 0),
        'ROUGE2': metrics.get('rouge2', 0),
        'ROUGEL': metrics.get('rougeL', 0),
        'Faithfulness': metrics.get('faithfulness', 0),
        'Eval Time (s)': metrics.get('eval_time_seconds', 0),
        'Throughput (samples/s)': metrics.get('samples_per_second', 0)
    }
    report_data.append(row)

report_df = pd.DataFrame(report_data)

# Save as CSV
output_path = Path("../experiments/batch_evaluation_test")
output_path.mkdir(parents=True, exist_ok=True)

csv_path = output_path / "batch_comparison_report.csv"
report_df.to_csv(csv_path, index=False)

print(f"\n✓ Report saved to: {csv_path}")
print("\n" + report_df.to_string(index=False))


In [ ]:
# Analyze distillation impact (if baseline and distilled models available)
print("\n" + "=" * 70)
print("DISTILLATION IMPACT ANALYSIS")
print("=" * 70)

# Create a dict for easier lookup
batch_results_dict = {r['model_name']: r['metrics'] for r in batch_evaluator.results}

# Check if we have both baseline and distilled models
distilled_models = [m for m in models if 'distilled' in m.lower()]
baseline_models = [m for m in models if 'baseline' in m.lower() or 'untrained' in m.lower()]

if distilled_models and baseline_models:
    print(f"\nFound {len(baseline_models)} baseline(s) and {len(distilled_models)} distilled model(s)")
    
    # Compare each pair
    improvements = []
    for baseline in baseline_models:
        for distilled in distilled_models:
            # Check if they're the same model size
            if any(x in baseline for x in ['small', 'base', 'large']):
                model_size = next(x for x in ['small', 'base', 'large'] if x in baseline)
                if model_size in distilled:
                    baseline_acc = batch_results_dict[baseline].get('label_accuracy', 0)
                    distilled_acc = batch_results_dict[distilled].get('label_accuracy', 0)
                    improvement = (distilled_acc - baseline_acc) / max(baseline_acc, 0.001) * 100
                    
                    improvements.append({
                        'model_size': model_size,
                        'baseline': baseline,
                        'distilled': distilled,
                        'baseline_acc': baseline_acc,
                        'distilled_acc': distilled_acc,
                        'improvement_pct': improvement
                    })
    
    # Display improvements
    if improvements:
        print("\nDistillation Improvements:")
        for imp in improvements:
            print(f"\n{imp['model_size'].upper()}:")
            print(f"  Baseline: {imp['baseline_acc']:.4f}")
            print(f"  Distilled: {imp['distilled_acc']:.4f}")
            print(f"  Improvement: {imp['improvement_pct']:+.2f}%")
else:
    print("\n💡 To analyze distillation impact, train models with both baseline and distilled variants")
    print("   and add them to the batch evaluator with 'baseline' and 'distilled' in their names.")


## 10. Model Size Degradation Analysis

Systematic analysis of performance degradation across FLAN-T5 model sizes (small, base, large).

In [ ]:
# Comprehensive model size comparison
print("=" * 70)
print("MODEL SIZE DEGRADATION ANALYSIS")
print("=" * 70)

model_sizes = ["small", "base", "large"]
degradation_results = []

for size in model_sizes:
    print(f"\n{'='*70}")
    print(f"Evaluating FLAN-T5-{size.upper()}")
    print(f"{'='*70}")
    
    # Create model
    model_name = f"google/flan-t5-{size}"
    student_config_size = StudentConfig(
        model_name=model_name,
        max_source_length=128,
        max_target_length=64,
        device=device,
        num_beams=4
    )
    
    student_size = StudentModel(student_config_size)
    
    # Get model info
    model_info = student_size.get_model_info()
    memory_info = student_size.get_memory_footprint()
    
    print(f"\nModel Information:")
    print(f"  Parameters: {model_info['parameters']:,}")
    print(f"  Memory: {memory_info['total_mb']:.2f} MB")
    print(f"  Layers: {model_info['encoder_layers']}")
    
    # Evaluate
    eval_config_size = EvaluationConfig(
        metrics_config=MetricsConfig(
            compute_rouge=True,
            compute_bertscore=False,
            compute_faithfulness=True
        ),
        num_beams=4,
        max_length=64,
        save_predictions=False,
        output_dir=f"../experiments/degradation_{size}_test"
    )
    
    evaluator_size = Evaluator(student_size, eval_config_size)
    
    # Time the evaluation
    start_time = time.time()
    results_size = evaluator_size.evaluate(test_loader, split_name=f"t5-{size}")
    eval_time = time.time() - start_time
    
    # Store comprehensive results
    degradation_results.append({
        'model_size': size,
        'model_name': model_name,
        'parameters': model_info['parameters'],
        'memory_mb': memory_info['total_mb'],
        'layers': model_info['encoder_layers'],
        'hidden_size': model_info['hidden_size'],
        'accuracy': results_size['metrics']['label_accuracy'],
        'rouge1': results_size['metrics']['rouge1'],
        'rouge2': results_size['metrics']['rouge2'],
        'rougeL': results_size['metrics']['rougeL'],
        'faithfulness': results_size['metrics']['faithfulness'],
        'eval_time': eval_time,
        'throughput': results_size['metrics']['samples_per_second']
    })
    
    print(f"\n✓ {size.upper()} evaluation complete")
    print(f"  Accuracy: {results_size['metrics']['label_accuracy']:.4f}")
    print(f"  ROUGE-L: {results_size['metrics']['rougeL']:.4f}")
    print(f"  Eval time: {eval_time:.2f}s")

print("\n" + "="*70)
print("DEGRADATION ANALYSIS COMPLETE")
print("="*70)

In [ ]:
# Create comprehensive degradation analysis DataFrame
degradation_df = pd.DataFrame(degradation_results)

print("\n" + "=" * 70)
print("DEGRADATION ANALYSIS SUMMARY")
print("=" * 70)
print("\n" + degradation_df.to_string(index=False))

# Calculate degradation percentages
base_idx = degradation_df[degradation_df['model_size'] == 'base'].index[0]
base_accuracy = degradation_df.loc[base_idx, 'accuracy']

print("\n" + "=" * 70)
print("DEGRADATION vs BASE MODEL")
print("=" * 70)

for _, row in degradation_df.iterrows():
    if row['model_size'] != 'base':
        acc_degradation = ((row['accuracy'] - base_accuracy) / base_accuracy) * 100
        rouge_degradation = ((row['rougeL'] - degradation_df.loc[base_idx, 'rougeL']) / degradation_df.loc[base_idx, 'rougeL']) * 100
        param_ratio = (row['parameters'] / degradation_df.loc[base_idx, 'parameters']) * 100
        
        print(f"\n{row['model_size'].upper()} vs BASE:")
        print(f"  Parameters: {param_ratio:.1f}% of base ({row['parameters']:,})")
        print(f"  Accuracy degradation: {acc_degradation:+.2f}%")
        print(f"  ROUGE-L degradation: {rouge_degradation:+.2f}%")
        print(f"  Speed ratio: {degradation_df.loc[base_idx, 'throughput'] / row['throughput']:.2f}x")
        print(f"  Memory ratio: {row['memory_mb'] / degradation_df.loc[base_idx, 'memory_mb']:.2f}x")

In [ ]:
# Visualize degradation curves
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: Accuracy vs Parameters
axes[0, 0].plot(degradation_df['parameters']/1e6, degradation_df['accuracy'], 
                marker='o', linewidth=3, markersize=12, color='#e74c3c')
axes[0, 0].set_xlabel('Parameters (Millions)')
axes[0, 0].set_ylabel('Label Accuracy')
axes[0, 0].set_title('Accuracy Degradation vs Model Size')
axes[0, 0].grid(True, alpha=0.3)
for x, y, label in zip(degradation_df['parameters']/1e6, degradation_df['accuracy'], degradation_df['model_size']):
    axes[0, 0].text(x, y + 0.01, label, ha='center', fontsize=10, fontweight='bold')

# Plot 2: ROUGE-L vs Parameters
axes[0, 1].plot(degradation_df['parameters']/1e6, degradation_df['rougeL'], 
                marker='s', linewidth=3, markersize=12, color='#3498db')
axes[0, 1].set_xlabel('Parameters (Millions)')
axes[0, 1].set_ylabel('ROUGE-L')
axes[0, 1].set_title('ROUGE-L Degradation vs Model Size')
axes[0, 1].grid(True, alpha=0.3)
for x, y, label in zip(degradation_df['parameters']/1e6, degradation_df['rougeL'], degradation_df['model_size']):
    axes[0, 1].text(x, y + 0.01, label, ha='center', fontsize=10, fontweight='bold')

# Plot 3: Faithfulness vs Parameters
axes[0, 2].plot(degradation_df['parameters']/1e6, degradation_df['faithfulness'], 
                marker='^', linewidth=3, markersize=12, color='#2ecc71')
axes[0, 2].set_xlabel('Parameters (Millions)')
axes[0, 2].set_ylabel('Faithfulness')
axes[0, 2].set_title('Faithfulness vs Model Size')
axes[0, 2].grid(True, alpha=0.3)
for x, y, label in zip(degradation_df['parameters']/1e6, degradation_df['faithfulness'], degradation_df['model_size']):
    axes[0, 2].text(x, y + 0.01, label, ha='center', fontsize=10, fontweight='bold')

# Plot 4: Throughput vs Parameters
axes[1, 0].plot(degradation_df['parameters']/1e6, degradation_df['throughput'], 
                marker='d', linewidth=3, markersize=12, color='#9b59b6')
axes[1, 0].set_xlabel('Parameters (Millions)')
axes[1, 0].set_ylabel('Throughput (samples/sec)')
axes[1, 0].set_title('Inference Speed vs Model Size')
axes[1, 0].grid(True, alpha=0.3)
for x, y, label in zip(degradation_df['parameters']/1e6, degradation_df['throughput'], degradation_df['model_size']):
    axes[1, 0].text(x, y + 0.5, label, ha='center', fontsize=10, fontweight='bold')

# Plot 5: Memory vs Parameters
axes[1, 1].plot(degradation_df['parameters']/1e6, degradation_df['memory_mb'], 
                marker='p', linewidth=3, markersize=12, color='#f39c12')
axes[1, 1].set_xlabel('Parameters (Millions)')
axes[1, 1].set_ylabel('Memory (MB)')
axes[1, 1].set_title('Memory Footprint vs Model Size')
axes[1, 1].grid(True, alpha=0.3)
for x, y, label in zip(degradation_df['parameters']/1e6, degradation_df['memory_mb'], degradation_df['model_size']):
    axes[1, 1].text(x, y + 20, label, ha='center', fontsize=10, fontweight='bold')

# Plot 6: Efficiency (Accuracy per Parameter)
efficiency = degradation_df['accuracy'] / (degradation_df['parameters'] / 1e6)
axes[1, 2].bar(degradation_df['model_size'], efficiency, 
               color=['#3498db', '#e74c3c', '#2ecc71'], alpha=0.7, edgecolor='black')
axes[1, 2].set_xlabel('Model Size')
axes[1, 2].set_ylabel('Accuracy per Million Parameters')
axes[1, 2].set_title('Parameter Efficiency')
axes[1, 2].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(efficiency):
    axes[1, 2].text(i, v + 0.0001, f'{v:.5f}', ha='center', fontsize=10, fontweight='bold')

plt.suptitle('Model Size Degradation Analysis: FLAN-T5 Family', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Performance-Efficiency Trade-off Analysis
print("=" * 70)
print("PERFORMANCE-EFFICIENCY TRADE-OFF")
print("=" * 70)

# Calculate compression ratio vs performance retention
base_params = degradation_df.loc[degradation_df['model_size'] == 'base', 'parameters'].values[0]
base_acc = degradation_df.loc[degradation_df['model_size'] == 'base', 'accuracy'].values[0]

tradeoff_data = []
for _, row in degradation_df.iterrows():
    compression_ratio = base_params / row['parameters']
    accuracy_retention = (row['accuracy'] / base_acc) * 100
    rouge_retention = (row['rougeL'] / degradation_df.loc[degradation_df['model_size'] == 'base', 'rougeL'].values[0]) * 100
    speedup = row['throughput'] / degradation_df.loc[degradation_df['model_size'] == 'base', 'throughput'].values[0]
    
    tradeoff_data.append({
        'model': row['model_size'],
        'compression_ratio': compression_ratio,
        'accuracy_retention_%': accuracy_retention,
        'rouge_retention_%': rouge_retention,
        'speedup': speedup
    })

tradeoff_df = pd.DataFrame(tradeoff_data)

print("\n" + tradeoff_df.to_string(index=False))

# Visualize trade-off
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Accuracy Retention vs Compression
colors_tradeoff = ['#3498db', '#e74c3c', '#2ecc71']
for idx, row in tradeoff_df.iterrows():
    axes[0].scatter(row['compression_ratio'], row['accuracy_retention_%'], 
                    s=300, alpha=0.7, color=colors_tradeoff[idx], edgecolor='black', linewidth=2)
    axes[0].text(row['compression_ratio'], row['accuracy_retention_%'] + 1, 
                 row['model'], ha='center', fontsize=12, fontweight='bold')

axes[0].set_xlabel('Compression Ratio (vs Base)', fontsize=12)
axes[0].set_ylabel('Accuracy Retention (%)', fontsize=12)
axes[0].set_title('Compression vs Performance Trade-off', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=100, color='red', linestyle='--', alpha=0.5, label='Base Performance')
axes[0].axhline(y=90, color='orange', linestyle='--', alpha=0.5, label='90% Retention')
axes[0].legend()

# Plot 2: Speedup vs Accuracy Retention
for idx, row in tradeoff_df.iterrows():
    axes[1].scatter(row['speedup'], row['accuracy_retention_%'], 
                    s=300, alpha=0.7, color=colors_tradeoff[idx], edgecolor='black', linewidth=2)
    axes[1].text(row['speedup'], row['accuracy_retention_%'] + 1, 
                 row['model'], ha='center', fontsize=12, fontweight='bold')

axes[1].set_xlabel('Speedup (vs Base)', fontsize=12)
axes[1].set_ylabel('Accuracy Retention (%)', fontsize=12)
axes[1].set_title('Speed vs Performance Trade-off', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=100, color='red', linestyle='--', alpha=0.5, label='Base Performance')
axes[1].axhline(y=90, color='orange', linestyle='--', alpha=0.5, label='90% Retention')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("KEY FINDINGS:")
print("="*70)

# Find best compression with minimal degradation
best_small = tradeoff_df[tradeoff_df['model'] == 'small'].iloc[0]
print(f"\nSMALL Model Trade-off:")
print(f"  • {best_small['compression_ratio']:.1f}x compression")
print(f"  • {best_small['accuracy_retention_%']:.1f}% accuracy retention")
print(f"  • {best_small['speedup']:.2f}x faster inference")
print(f"  • Recommendation: {'EXCELLENT' if best_small['accuracy_retention_%'] >= 95 else 'GOOD' if best_small['accuracy_retention_%'] >= 90 else 'ACCEPTABLE'} for production")

In [ ]:
# Save degradation analysis results
degradation_output_dir = Path("../experiments/degradation_analysis")
degradation_output_dir.mkdir(parents=True, exist_ok=True)

degradation_df.to_csv(degradation_output_dir / "model_size_comparison.csv", index=False)
tradeoff_df.to_csv(degradation_output_dir / "tradeoff_analysis.csv", index=False)

print(f"\n✓ Degradation analysis results saved to {degradation_output_dir}")
print("\n" + "="*70)
print("🎯 DEGRADATION ANALYSIS COMPLETE")
print("="*70)
print("\n📊 Summary:")
print(f"  • Evaluated {len(model_sizes)} model sizes")
print(f"  • Parameter range: {degradation_df['parameters'].min()/1e6:.1f}M - {degradation_df['parameters'].max()/1e6:.1f}M")
print(f"  • Accuracy range: {degradation_df['accuracy'].min():.3f} - {degradation_df['accuracy'].max():.3f}")
print(f"  • Best efficiency: {tradeoff_df.loc[tradeoff_df['accuracy_retention_%'].idxmax(), 'model']}")
print("\n💡 Use these insights to select optimal model size for your use case!")